# PCA and shit 

In [73]:
import h5py 
import numpy as np
import pandas as pd
import seaborn as sns

import os 
import random
random.seed(42)

from sklearn.linear_model import Lasso
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
from sklearn.metrics import roc_curve

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import preprocessing


In [74]:
data_parent_dir = "./data_fixed_crop_w_mask"
precomputed_path = "features_matrix.csv"

In [75]:
precomputed_df = pd.read_csv(precomputed_path)
feature_names = list(precomputed_df)[1:]
precomputed_fts = precomputed_df.values[:,1:]
precomputed_fts =np.array(precomputed_fts,dtype=np.float32)
print(precomputed_fts[0])

[ 9.8299998e-01  1.2056100e+05  5.4922956e-01  1.4699945e-01
  2.7999999e-03  1.6000000e-03  5.3562564e-01  9.9140000e-01
  9.9940002e-01  1.9901167e+02  1.4164154e+02  8.8326851e+01
 -1.4469041e+00  2.9172947e+03  7.8043795e-01  8.2708019e-01
  1.2968200e+01  9.7119999e-01]


We're going to do an initial zero-centering

In [76]:
precomputed_fts -= np.mean(precomputed_fts, axis = 0)
precomputed_fts /= np.std(precomputed_fts, axis=0)


In [77]:
pca = PCA()
pca.fit(precomputed_fts)

PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [78]:
print(pca.explained_variance_ratio_)  
print(pca.components_[0])

[4.4473657e-01 1.6824159e-01 9.1169290e-02 5.7367690e-02 5.6748547e-02
 5.2320685e-02 3.6198188e-02 3.1822890e-02 2.2656877e-02 1.7601196e-02
 9.3034180e-03 7.1804966e-03 2.3674807e-03 1.1376145e-03 8.7159796e-04
 2.6753976e-04 6.5508975e-06 1.8152512e-06]
[-0.33582258  0.3129383  -0.04206447 -0.01204051  0.31197217  0.3208049
  0.00518964 -0.33550972 -0.30273733  0.21556541  0.16609216  0.02282244
  0.00318173  0.30442756  0.21560022 -0.05786494  0.3394732   0.22484674]


In [79]:
transformed = pca.transform(precomputed_fts)[:,:3]
transformed.shape

(1599, 3)

In [80]:
def reduce_dimensionality(raw_data, new_dims=3):
    pca = PCA()
    pca.fit(raw_data)
    print("variance explained by each principal component")
    print(pca.explained_variance_ratio_)  
    print("the first principal component")
    print(pca.components_[0])
    return pca.transform(precomputed_fts[:,:3])

In [81]:
def get_image_features(paths):
    data = []
    label = []
    name = []
    for first_path in paths:
        if first_path[0] == '.': # check for random dot files that come up :( )
            continue
        local_dir = os.path.join(data_parent_dir, first_path)
        for image in os.listdir(local_dir):
            hf = h5py.File(os.path.join(local_dir, image), 'r')
            data.append(np.array(hf.get('data')))
            label.append(np.array(hf.get('label')).item(0))
            name.append(np.array(hf.get('name')))
    d = {'pixel_data':data, 'label':label, 'name':name}
    df = pd.DataFrame(data=d)
    return df

In [82]:
all_paths = os.listdir(data_parent_dir)
image_df = get_image_features(all_paths)

In [89]:
semantics_path = "mass_case_description_train_set.csv"
semantic_df = pd.read_csv(semantics_path)
semantic_df.dropna(inplace=True)

In [90]:
semantic_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1273 entries, 0 to 1317
Data columns (total 13 columns):
patient_id        1273 non-null object
breast_density    1273 non-null int64
side              1273 non-null object
view              1273 non-null object
abn_num           1273 non-null int64
mass_shape        1273 non-null object
mass_margins      1273 non-null object
assessment        1273 non-null int64
pathology         1273 non-null object
subtlety          1273 non-null int64
od_img_path       1273 non-null object
od_crop_path      1273 non-null object
mask_path         1273 non-null object
dtypes: int64(4), object(9)
memory usage: 139.2+ KB


In [91]:
semantic_feature_names = ['breast_density', 'abn_num', 'mass_shape', 'mass_margins', 'assessment']
semantic_label_name = ['pathology']

In [93]:
for feature in semantic_feature_names:
    le = preprocessing.LabelEncoder()
    #print(semantic_df[feature].values)
    #print(semantic_df[semantic_df[feature].isnull() == True])
    le.fit(list(semantic_df[feature].astype(str)))
    print(le.classes_)
    semantic_df[feature] = le.transform(semantic_df[feature])

['1' '2' '3' '4']
['1' '2' '3' '4' '5' '6']
['ARCHITECTURAL_DISTORTION' 'ASYMMETRIC_BREAST_TISSUE'
 'FOCAL_ASYMMETRIC_DENSITY' 'IRREGULAR'
 'IRREGULAR-ARCHITECTURAL_DISTORTION' 'IRREGULAR-FOCAL_ASYMMETRIC_DENSITY'
 'LOBULATED' 'LOBULATED-ARCHITECTURAL_DISTORTION' 'LOBULATED-IRREGULAR'
 'LOBULATED-LYMPH_NODE' 'LOBULATED-OVAL' 'LYMPH_NODE' 'OVAL'
 'OVAL-LYMPH_NODE' 'ROUND' 'ROUND-IRREGULAR-ARCHITECTURAL_DISTORTION'
 'ROUND-LOBULATED' 'ROUND-OVAL']
['CIRCUMSCRIBED' 'CIRCUMSCRIBED-ILL_DEFINED'
 'CIRCUMSCRIBED-MICROLOBULATED' 'CIRCUMSCRIBED-OBSCURED' 'ILL_DEFINED'
 'ILL_DEFINED-SPICULATED' 'MICROLOBULATED' 'MICROLOBULATED-ILL_DEFINED'
 'MICROLOBULATED-ILL_DEFINED-SPICULATED' 'MICROLOBULATED-SPICULATED'
 'OBSCURED' 'OBSCURED-ILL_DEFINED' 'OBSCURED-ILL_DEFINED-SPICULATED'
 'OBSCURED-SPICULATED' 'SPICULATED']
['0' '1' '2' '3' '4' '5']


In [94]:
enc = preprocessing.OneHotEncoder()
enc.fit(semantic_df[semantic_feature_names])

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [95]:
print(semantic_df[semantic_feature_names[2]].values)

[ 4  4  0 ... 14  3  3]
